In [1]:
import os, sys
sys.path.append("../")
sys.path.append("./")
sys.path.append("./src/network")
sys.path.append("./src")

import numpy as np
import string
import cv2
import numpy as np
import random
from random import choices
import matplotlib.pyplot as plt

import torch
from torch import optim
from torchvision import transforms
from torchvision.transforms import v2
import torch.utils.data as D

from src.network.gen_model.gen_model import GenModel_FC
from src.data.tokenizer import Tokenizer
from src.data.data_loader import IAM_data
from src.data.reader import read_iam

['.ipynb_checkpoints', '.RData', '.Rhistory', '220[1].jpg', 'Aangepaste Office-sjablonen', 'ADS_programming_report_template.pdf', "Assassin's Creed Origins", 'BioWare', 'desktop.ini', 'Downloads', 'FeedbackHub', 'GitHub', 'Graphics', 'Mijn afbeeldingen', 'Mijn muziek', "Mijn video's", 'My Games', 'Need for Speed Heat', 'Python Notebooks', 'Python Scripts', 'Scripts', 'Untitled.ipynb', 'Visual Studio 2019']
c:\Users\Lisa\Documents\GitHub\handwriting-recognition
c:\Users\Lisa\Documents\GitHub\handwriting-recognition


In [2]:
def read_data(folder, max_word_len, synth):
    """Get data paths and labels (with max_word_len) of images in folder."""

    partitions = ['train', 'valid']
    dataset = {}
    wid_dict = {}
    for partition in partitions:
        lens = {}
        print(partition)
        dataset[partition] = []
        wid_dict[partition] = {}

        text_file = os.path.join(folder, f"ground_truth_{partition}_filtered.txt")

        with open(text_file, encoding='utf-8') as data_file:
            lines = data_file.read().splitlines()

        for line in lines:
            line_split = line.split(' ')
            wid = None

            if not set(string.digits).isdisjoint(set(line_split[-1])):
                wid = line_split[-1]
                line_split = line_split[:-1]
                
            if len(line_split) > 2:
                img_path, gt_label = line_split[0], ''.join(line_split[1:])
            else:
                img_path, gt_label = line_split[0], line_split[1]
                
            if len(gt_label) > max_word_len:
                continue

            if len(gt_label) in lens.keys():
                lens[len(gt_label)] += 1
            else:
                lens[len(gt_label)] = 1
            
            img_path = img_path.replace("/", "\\")

            if wid is not None:
                if wid not in wid_dict[partition].keys():
                    wid_dict[partition][wid] = []
                
                wid_dict[partition][wid].append((img_path, gt_label))
            
            if synth:
                img_path_split = img_path.split('\\')
                img_path = os.path.join(img_path_split[0], img_path_split[1], img_path_split[2], img_path_split[3], "valid_gen", img_path_split[-1])
            else:
                img_path = img_path[1:]

            # img_path = os.path.join(folder, partition, img_path)
            dataset[partition].append((img_path[1:], gt_label, wid))
        print(f"number of words in {partition}: {len(dataset[partition])}")
        print(f"number of wids in {partition}: {len(wid_dict[partition].keys())}")
    
        lens = dict(sorted(lens.items()))

        print(f"Number of words per word length")
        for key in lens.keys():
            print(key, lens[key])

    # Adding training samples to wid dictionaries for validation and testing sets
    for wid in wid_dict['train'].keys():
        if wid in wid_dict['valid'].keys():
            wid_dict['valid'][wid] += wid_dict['train'][wid]

    return dataset['train'], dataset['valid'],  wid_dict['train'], wid_dict['valid']

In [3]:
charset_base = string.ascii_lowercase + string.ascii_uppercase
max_text_length = 25
tokenizer = Tokenizer(chars=charset_base, max_text_length=max_text_length, ctc=False)

In [8]:
gen_epochs = 2500
gen_model = GenModel_FC(tokenizer.maxlen, tokenizer.vocab_size, tokenizer.PAD).cuda()
gen_model.load_state_dict(torch.load(f'./src/network/gen_model/gen_model-{gen_epochs}-25ch-corr-half.model')) #load model
gen_model.eval()

for param in gen_model.parameters():
    param.requires_grad = False

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = (64, 216, 1)
num_style_imgs = 25 # num imgs for generator to extract style from
batch_size = 48
dataset = "iam_gan"
dataset_path = os.path.join("data", dataset, "words")
max_word_len = 10

In [6]:
data_train, data_valid, data_test, oov_data_train, oov_data_valid, oov_data_test, wid_train, wid_valid, wid_test, oov_wid_train, oov_wid_valid, oov_wid_test = read_iam(dataset_path, 10, synth=False)
        
oov_data_valid = IAM_data(oov_data_valid, input_size=input_size, tokenizer=tokenizer, num_images=num_style_imgs, wids=oov_wid_valid)
oov_valid_loader = torch.utils.data.DataLoader(oov_data_valid, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=4)

data_test = IAM_data(data_test, input_size=input_size, tokenizer=tokenizer, num_images=num_style_imgs, wids=wid_test)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=4)

oov_data_test = IAM_data(oov_data_test, input_size=input_size, tokenizer=tokenizer, num_images=num_style_imgs, wids=oov_wid_test)
oov_test_loader = torch.utils.data.DataLoader(oov_data_test, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=4)

data_train = IAM_data(data_train, input_size=input_size, tokenizer=tokenizer, num_images=num_style_imgs, wids=wid_train)
train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=1)

oov_data_train = IAM_data(oov_data_train, input_size=input_size, tokenizer=tokenizer, num_images=num_style_imgs, wids=wid_train)
oov_train_loader = torch.utils.data.DataLoader(oov_data_train, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=1)

data_valid = IAM_data(data_valid, input_size=input_size, tokenizer=tokenizer, num_images=num_style_imgs, wids=wid_valid)
valid_loader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=1)

train
number of words in train: 19744
number of wids in train: 500
Number of words per word length
2 4645
3 5617
4 4030
5 2404
6 1608
7 1440
valid
number of words in valid: 4105
number of wids in valid: 499
Number of words per word length
2 954
3 1135
4 806
5 513
6 373
7 324
test
number of words in test: 4357
number of wids in test: 500
Number of words per word length
2 978
3 1251
4 898
5 552
6 371
7 307
oov_train
number of words in oov_train: 1324
number of wids in oov_train: 306
Number of words per word length
2 13
3 76
4 257
5 334
6 358
7 286
oov_valid
number of words in oov_valid: 866
number of wids in oov_valid: 329
Number of words per word length
2 9
3 58
4 181
5 218
6 230
7 170
oov_test
number of words in oov_test: 1164
number of wids in oov_test: 352
Number of words per word length
2 12
3 63
4 223
5 293
6 320
7 253


# Recreate a dataset of IAM-HTR

In [12]:
for batch in oov_test_loader:
    imgs, gen_imgs, gt_labels, wid, img_paths = batch
    gt_labels = torch.nn.functional.one_hot(gt_labels.long(), 56).float()

    gen_imgs = gen_imgs.to(device).squeeze(2)
    gt_labels = gt_labels.to(device)

    imgs = imgs.numpy()
    imgs = 255. - (((imgs * 0.5) + 0.5) * 255)
        
    synth_imgs = gen_model(gen_imgs, gt_labels).detach()
    synth_imgs = 255. - (((synth_imgs * 0.5) + 0.5) * 255)
    synth_imgs = synth_imgs.cpu().numpy()

    for idx in range(len(img_paths)):

        path_split = img_paths[idx].split('\\')
        img_name = path_split[-1][:-4]  # Removal of .png 
        img_name = img_name + '_synth.png'
        new_img_path = os.path.join('.', path_split[1], path_split[2], path_split[3], path_split[4], path_split[5], path_split[6], img_name)

        # path_split = img_paths[idx].split('.png')[0]
        # new_img_path = path_split + '_synth.png'
        cv2.imwrite(new_img_path, synth_imgs[idx][0])


In [22]:
for batch in oov_test_loader:
    imgs, gen_imgs, gt_labels, wid, img_paths = batch
    gt_labels = torch.nn.functional.one_hot(gt_labels.long(), 56).float()

    gen_imgs = gen_imgs.to(device).squeeze(2)
    gt_labels = gt_labels.to(device)

    imgs = imgs.numpy()
    imgs = 255. - (((imgs * 0.5) + 0.5) * 255)

        
    synth_imgs = gen_model(gen_imgs, gt_labels).detach()
    synth_imgs = 255. - (((synth_imgs * 0.5) + 0.5) * 255)
    synth_imgs = synth_imgs.cpu().numpy()

    for idx in range(len(img_paths)):

        path_split = img_paths[idx].split('\\')
        img_name = path_split[-1][:-4]  # remove .png 
        img_name = img_name + '_synth.png'
        new_img_path = os.path.join('.', path_split[1], path_split[2], path_split[3], path_split[4], path_split[5], path_split[6], img_name)

        # path_split = img_paths[idx].split('.png')[0]
        # new_img_path = path_split + '_synth.png'
        cv2.imwrite(new_img_path, synth_imgs[idx][0])

# Recreate the IAM-GEN validation set

In [10]:
data_train, data_valid, wid_train, wid_valid = read_data(dataset_path, 10, synth=False)

data_valid = IAM_data(data_valid, input_size=input_size, tokenizer=tokenizer, num_images=num_style_imgs, wids=wid_valid)
valid_loader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=1)

train
number of words in train: 22117
number of wids in train: 339
Number of words per word length
2 4817
3 5670
4 4367
5 2981
6 2242
7 2040
valid
number of words in valid: 9178
number of wids in valid: 161
Number of words per word length
2 1860
3 2502
4 1937
5 1261
6 867
7 751


In [11]:
for batch in valid_loader:
    imgs, gen_imgs, gt_labels, wid, img_paths = batch
    gt_labels = torch.nn.functional.one_hot(gt_labels.long(), 56).float()

    gen_imgs = gen_imgs.to(device).squeeze(2)
    gt_labels = gt_labels.to(device)

    imgs = imgs.numpy()
    imgs = 255. - (((imgs * 0.5) + 0.5) * 255)

        
    synth_imgs = gen_model(gen_imgs, gt_labels).detach()
    synth_imgs = 255. - (((synth_imgs * 0.5) + 0.5) * 255)
    synth_imgs = synth_imgs.cpu().numpy()

    for idx in range(len(img_paths)):
        
        path_split = img_paths[idx].split('\\')
        img_name = path_split[-1][:-4] # Removal of .png 
        new_dir = os.path.join("data", dataset, "words", "valid_gen_2500")
        os.makedirs(new_dir, exist_ok=True)
        new_img_path = os.path.join(new_dir, img_name + f'.png')

        cv2.imwrite(new_img_path, synth_imgs[idx][0])

        # Uncomment to save images in folders per wid
        # new_dir = os.path.join("data", dataset, "words", "valid_gen_1750", wid[idx])
        # os.makedirs(new_dir, exist_ok=True)
        # new_img_path = os.path.join(new_dir, img_name + f'.png')
        # cv2.imwrite(new_img_path, synth_imgs[idx][0])

        # if i == 0:
        #     new_dir = os.path.join("data", dataset, "words", "valid", wid[idx])
        #     os.makedirs(new_dir, exist_ok=True)
        #     new_img_path = os.path.join(new_dir, path_split[-1])
        #     cv2.imwrite(new_img_path, imgs[idx][0])



